# **MediaPipe**

**Refrence:** https://ai.google.dev/edge/mediapipe/solutions/guide

**MediaPipe** is an open-source framework developed by Google that provides pre-built, high-performance machine learning solutions for computer vision.

Think of it this way:

* **OpenCV** gives you the "tools" to process images (blur, resize, threshold).
* **MediaPipe** gives you **complete, pre-trained pipelines** to understand what is *in* the image (hands, faces, poses).

### Key Features

It is famous for being **extremely fast** (real-time) even on mobile devices and CPUs because it is highly optimized.

Common solutions include:

* **Hands:** Tracks 21 3D landmarks on a hand (knuckles, fingertips).
* **Pose:** Tracks full-body skeletal points (shoulders, elbows, knees).
* **Face Mesh:** Maps 468 points on a face (lips, eyes, eyebrows).
* **Selfie Segmentation:** Separates a person from the background (like Zoom background blur).

### How it is used in a CV Project

You typically use **OpenCV** to capture the video, and **MediaPipe** to analyze it.

**The Workflow:**

1. **Capture:** Use OpenCV (`cv2.VideoCapture`) to get a frame.
2. **Color Convert:** MediaPipe expects **RGB** images, but OpenCV loads in **BGR**, so you must convert it (`cv2.cvtColor`).
3. **Process:** Pass the image to a MediaPipe solution (e.g., `hands.process(image)`).
4. **Extract Data:** MediaPipe returns **"Landmarks"** (x, y, z coordinates).
* Example: "Index finger tip is at x: 0.5, y: 0.3".


5. **Draw:** You can use these coordinates to draw lines or control a game/mouse.

**Simple Logic Example:**

```python
import cv2
import mediapipe as mp

# 1. Initialize
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# 2. Capture
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

# 3. Process
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
results = hands.process(rgb_frame)

# 4. Use Results
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        # Get coordinates of the index finger tip (Point 8)
        print(hand_landmarks.landmark[8]) 

```

---
---

### 1. The Coding Style (`with ... as`)

This is called a **Context Manager**.

* **What it does:** It automatically handles "cleanup" for you.
* **Why use it:** When the code block finishes (or if it crashes), Python automatically closes the `face_detection` model to free up memory. It's safer than manually opening and closing it.

### 2. The Parameters

* **`model_selection=1` (Range Setting):**
* **0:** Use for faces **close** to the camera (within 2 meters, e.g., phone selfies).
* **1:** Use for faces **far** from the camera (2 to 5 meters, e.g., full-body shots).


* **`min_detection_confidence=0.5` (Strictness):**
* This is the "Trust Threshold."
* **0.5 (50%):** The AI must be at least 50% sure it found a face before showing it to you.
* *Higher (e.g., 0.9):* Very strict, fewer mistakes, but might miss some faces.
* *Lower (e.g., 0.1):* Very loose, catches everything, but might think a lamp is a face.

---
---

# **MediaPipe on Windows Fix Guide**

### **The "MediaPipe on Windows" Fix Guide**

**Step 0: The File Name Rule (Critical)**

Before doing anything, make sure you **do not** have any file named `mediapipe.py` in your folder. If you do, rename it to `my_test.py`.

**Step 1: Force Python 3.11**

Run this command to switch `uv` away from Python 3.12 (which breaks MediaPipe).

Bash

```
uv python pin 3.11
```

**Step 2: Install The "Golden Combo"**

Run this **single command** to install all the compatible library versions at once. This prevents the "Dependency Hell" errors.

Bash

```
uv add "numpy<2" "opencv-python<4.10" "protobuf<4" "mediapipe==0.10.9"
```

- **Why this works:**
    
    - `numpy<2`: MediaPipe crashes on the new NumPy 2.0.
        
    - `opencv-python<4.10`: Old OpenCV is needed to work with old NumPy.
        
    - `protobuf<4`: MediaPipe needs the old Protobuf 3 to talk to C++.
        
    - `mediapipe==0.10.9`: The most stable version for Windows right now.
        

**Step 3: Sync & Run**

Finalize the setup and run the code.

Bash

```
uv sync
uv run image.py
```

---
---

# **relative to absolute conversion**

This step converts **Percentages** (0.0 to 1.0) into **Pixels** (0 to 1920, etc.).

Here is the breakdown:

1. **MediaPipe speaks in Percentages (Relative):**
* It returns values like `xmin = 0.5`.
* This means: "The face starts at **50%** of the image width."
* It doesn't know (or care) how big your image actually is in pixels.


2. **OpenCV needs Pixels (Absolute):**
* To draw a rectangle, OpenCV needs exact numbers like `x = 960`.
* It cannot draw at coordinate "0.5".


3. **The Math:**
* `x1 * W`  `0.5 * 1920` = **960 pixels**.
* `int(...)`  Rounds the decimal (e.g., 960.4) to a whole number (960) because pixels must be integers.



**In short:** You are translating "The middle of the screen" into "Pixel number 960."

---
---

# **argparse**

`argparse` is a built-in Python library used to write **command-line interfaces (CLIs)**.

In simple terms, it lets you control your code from the terminal (command prompt) without changing the code itself.

### Why do we use it?

Instead of hard-coding variables like `mode = 'video'` or changing the file path inside the script every time you want to run a different video, you can pass them as "arguments" when you run the command.

**Example without argparse:**
You have to open the file, find line 10, change `img_path = "photo.jpg"` to `video.mp4`, save, and run.

**Example with argparse:**
You just type this in the terminal:

```bash
python main.py --mode video --filePath my_video.mp4

```

### Key Features in Your Code

1. **`argparse.ArgumentParser()`**: This creates the "container" that listens for commands from the terminal.
2. **`args.add_argument(...)`**: This defines the rules.
* `--mode`: You defined a flag named "mode".
* `default='webcam'`: If the user doesn't type anything, the code assumes they want to use the webcam.


3. **`args.parse_args()`**: This is the "Go" button. It reads what the user typed in the terminal and stores the answers in the `args` variable.

### How it works in your project

Because of these lines:

```python
args.add_argument("--mode", default='webcam')
args.add_argument("--filePath", default=None)

```

You can now run your script in three different ways without changing a single line of code:

* **To run Webcam:**
`python main.py` (It uses the default 'webcam')
* **To run an Image:**
`python main.py --mode image --filePath ./assets/photo.jpg`
* **To run a Video:**
`python main.py --mode video --filePath ./assets/video.mp4`